In [ ]:
from pathlib import Path
import json
import os


def clean_and_rename(data_dir: Path):
    """
    Cleans JSON files by removing specified keys and renames them based on their original names.

    - Removes 'expect_url' and 'predict_url' from list of dictionaries in each JSON.
    - Renames files: adds '0916' prefix if filename ends with '2',
      and '095' prefix if it ends with '4'.
    """
    if not data_dir.is_dir():
        print(f"Error: Directory not found at {data_dir}")
        return

    print(f"Processing files in: {data_dir}")

    # Get a list of files first to avoid issues with renaming during iteration
    json_files = list(data_dir.glob('*.json'))

    for fp in json_files:
        try:
            # Read the JSON file
            with open(fp, 'r', encoding='utf-8') as f:
                content = json.load(f)

            # Ensure content is a list of dictionaries
            if not isinstance(content, list):
                print(f"Skipping {fp.name}: content is not a list.")
                continue

            # Remove specified keys
            keys_to_remove = ['expect_url', 'predict_url']
            for item in content:
                if isinstance(item, dict):
                    for key in keys_to_remove:
                        item.pop(key, None)

            # Write the cleaned content back to the original file
            with open(fp, 'w', encoding='utf-8') as f:
                json.dump(content, f, ensure_ascii=False, indent=2)

            print(f"Cleaned {fp.name}")

            # Rename the file based on rules
            stem = fp.stem
            new_name = fp.name
            if stem.endswith('2'):
                new_name = '0916' + fp.name
            elif stem.endswith('4'):
                new_name = '095' + fp.name

            if new_name != fp.name:
                new_path = fp.with_name(new_name)
                if not new_path.exists():
                    fp.rename(new_path)
                    print(f"Renamed {fp.name} to {new_name}")
                else:
                    print(
                        f"Skipping rename for {fp.name}: {new_name} already exists.")

        except json.JSONDecodeError:
            print(f"Skipping {fp.name}: Invalid JSON format.")
        except Exception as e:
            print(f"An error occurred while processing {fp.name}: {e}")


if __name__ == "__main__":
    # The script is in 'analyze', so we go up one level to the project root,
    # then into 'data/upload'.
    current_dir = Path(os.path.abspath('.')).parent
    data_directory = current_dir / 'data' / 'upload'

    clean_and_rename(data_directory.resolve())

    print("\nProcessing complete.")